### H2O

H2O - это еще один инструмент для машинного обучения, нацеленный на удобный интерфейс взаимодействия и возможность эффективного распределенного обучения. Официальный сайт - https://www.h2o.ai/

В Azure есть готовый компонент, который позволит развернуть кластер H2O по клику - https://portal.azure.com/#create/h2o-ai.h2o-sparklingwaterh2o-sparklingwater

Весь этот ноутбук запущен именно на таком кластере

In [1]:
%%configure -f
{
    "conf":{
        "spark.ext.h2o.announce.rest.url": "http://ed10-lsml-h.axexy4k5kbfuxexn2dl0fl3zub.dx.internal.cloudapp.net:5000/flows",
        "spark.jars":"/H2O-Sparkling-Water-files/sparkling-water-assembly-all.jar",
        "spark.submit.pyFiles":"/H2O-Sparkling-Water-files/pySparkling.zip",
        "spark.locality.wait":"3000",
        "spark.scheduler.minRegisteredResourcesRatio":"1",
        "spark.task.maxFailures":"1",
        "spark.yarn.am.extraJavaOption":"-XX:MaxPermSize=384m",
        "spark.yarn.max.executor.failures":"1",
        "maximizeResourceAllocation": "true"
    },
    "driverMemory":"21G",
    "executorMemory":"21G",
    "numExecutors":3
}

In [2]:
import pyspark
import os
os.environ["PYTHON_EGG_CACHE"] = "~/"
sc.addPyFile("wasb:///H2O-Sparkling-Water-files/pySparkling.zip") # For Azure DataLake replace wasb with adl

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1614009348742_0004,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.
/usr/bin/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')

In [3]:
import pysparkling, h2o

h2o_context = pysparkling.H2OContext.getOrCreate(sc)

Connecting to H2O server at http://10.0.0.13:54321 ... successful.
--------------------------  ------------------------------------------------------------------
H2O cluster uptime:         09 secs
H2O cluster timezone:       Etc/UTC
H2O data parsing timezone:  UTC
H2O cluster version:        3.26.0.5
H2O cluster version age:    1 year, 5 months and 6 days !!!
H2O cluster name:           sparkling-water-yarn_application_1614009348742_0004
H2O cluster total nodes:    3
H2O cluster free memory:    55.58 Gb
H2O cluster total cores:    24
H2O cluster allowed cores:  9
H2O cluster status:         accepting new members, healthy
H2O connection url:         http://10.0.0.13:54321
H2O connection proxy:
H2O internal security:      False
H2O API Extensions:         XGBoost, Algos, Amazon S3, AutoML, Core V3, TargetEncoder, Core V4
Python version:             2.7.12 final
--------------------------  ------------------------------------------------------------------

Sparkling Water Context:
 * Spa

In [5]:
h2o.cluster().show_status()

--------------------------  ------------------------------------------------------------------
H2O cluster uptime:         1 min 18 secs
H2O cluster timezone:       Etc/UTC
H2O data parsing timezone:  UTC
H2O cluster version:        3.26.0.5
H2O cluster version age:    1 year, 5 months and 6 days !!!
H2O cluster name:           sparkling-water-yarn_application_1614009348742_0004
H2O cluster total nodes:    3
H2O cluster free memory:    55.58 Gb
H2O cluster total cores:    24
H2O cluster allowed cores:  9
H2O cluster status:         locked, healthy
H2O connection url:         http://10.0.0.13:54321
H2O connection proxy:
H2O internal security:      False
H2O API Extensions:         XGBoost, Algos, Amazon S3, AutoML, Core V3, TargetEncoder, Core V4
Python version:             2.7.12 final
--------------------------  ------------------------------------------------------------------

In [10]:
%%local

! wget https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip
! unzip drugsCom_raw.zip
! hdfs dfs -rm -r /drugs/data || true
! hdfs dfs -mkdir -p /drugs/data

--2021-02-22 16:20:00--  https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42989872 (41M) [application/x-httpd-php]
Saving to: ‘drugsCom_raw.zip’

drugsCom_raw.zip    100%[===================>]  41.00M  91.2MB/s    in 0.4s    

2021-02-22 16:20:01 (91.2 MB/s) - ‘drugsCom_raw.zip’ saved [42989872/42989872]

Archive:  drugsCom_raw.zip
  inflating: drugsComTest_raw.tsv    
  inflating: drugsComTrain_raw.tsv   
rm: `/drugs/data': No such file or directory


In [11]:
%%bash

cat drugsComTrain_raw.tsv <(tail -n +2 drugsComTest_raw.tsv) | hdfs dfs -put - /drugs/data/drugs.tsv

In [12]:
%%local

! hdfs dfs -ls  -h /drugs/data

Found 1 items
-rw-r--r--   1 spark supergroup    107.2 M 2021-02-22 16:20 /drugs/data/drugs.tsv


Полезно заметить, что инструмент умеет самостоятельно понимать, какой формат данных вы читаете и сам его сразу же и парсить.

In [59]:
df = h2o.import_file("wasb:///drugs/data/drugs.tsv")

Parse progress: [#########################################################] 100%

In [60]:
df

  drugName  condition                           review                        rating                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               date  usefulCount          C7
----------  ----------------------------------  ----------------------------  -------------------------------------------------------------------------

Увы в Azure настолько старая версия кластера, что сюда даже tf-idf не завезли. Так что на этот раз мы с вами без текстов, но в реальной жизни просто разворачивайте последнюю версию кластера и будет вам счастье.

In [61]:
from h2o.information_retrieval.tf_idf import tf_idf

No module named information_retrieval.tf_idf
Traceback (most recent call last):
ImportError: No module named information_retrieval.tf_idf



In [62]:
df['drugName'] = df['drugName'].ascharacter().asfactor()
df['condition'] = df['condition'].ascharacter().asfactor()
df['usefulCount'] = df['usefulCount'].asnumeric()

In [63]:
df

  drugName  condition                           review                        rating                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               date    usefulCount    C7
----------  ----------------------------------  ----------------------------  -----------------------------------------------------------------------------

Говорим, что признаки, а что - целевая переменная в нашей таблице.

In [64]:
fcolumns = ["condition", "drugName", "rating"]
target = 'usefulCount'

In [65]:
train, valid= df.split_frame(ratios = [.7], seed = 7665)

Будем обучать градиентный бустинг. Так как это распределенный алгоритм на деревьях, то мы можем подтюнить параметры бинов (корзинок) для feature binning.

`nbins_cats` - это количество корзинок, которое необходимо выделить для работы с категориальными признаками.

`nbins` - количество корзинок для вещественных признаков.

In [69]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator

gbm = H2OGradientBoostingEstimator(nbins_cats=512, nbins=32, ntrees=13, learn_rate=0.09, seed=7896)

In [70]:
gbm.train(x = fcolumns, y = target, training_frame = train, validation_frame = valid)

gbm Model Build progress: [###############################################] 100%

In [71]:
gbm.r2(valid=True)

0.005611239991560324

Качество плохое, потому что мы не использовали текст. Чтобы как-то реабилитировать H2O в ваших глазах, давайте возьмем какой-нить датасет, где нет текста, а есть только категории и вещественные признаки и обучим бустинг там.

В качестве такого датасета возьмем датасет с airbnb. Нам будет необходимо по различным параметрам предсказывать стоимость аппартаментов.

In [72]:
%%local

! wget https://raw.githubusercontent.com/ADKosm/lsml-2021-public/main/data/airbnb-300k.tsv
! hdfs dfs -mkdir -p /airbnb
! hdfs dfs -put airbnb-300k.tsv /airbnb/data.tsv

--2021-02-22 18:16:13--  https://raw.githubusercontent.com/ADKosm/lsml-2021-public/main/data/airbnb-300k.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23835269 (23M) [text/plain]
Saving to: ‘airbnb-300k.tsv’

airbnb-300k.tsv     100%[===================>]  22.73M  --.-KB/s    in 0.1s    

2021-02-22 18:16:14 (189 MB/s) - ‘airbnb-300k.tsv’ saved [23835269/23835269]



In [73]:
df = h2o.import_file("wasb:///airbnb/data.tsv")

Parse progress: [#########################################################] 100%

In [74]:
df

  Price  City                        Zipcode  Last scraped         Country code    Property type    Room type        Bed type         Accommodates    Bathrooms    Bedrooms
-------  ------------------------  ---------  -------------------  --------------  ---------------  ---------------  -------------  --------------  -----------  ----------
     50  Paris                         75011  2017-04-06 00:00:00  FR              Apartment        Private room     Real Bed                    2            1           1
    125  Paris                         75011  2017-04-05 00:00:00  FR              Apartment        Entire home/apt  Real Bed                    4            1           1
     59  Paris                         75011  2017-04-05 00:00:00  FR              Apartment        Entire home/apt  Pull-out Sofa               2            1           0
     50  Paris                         75011  2017-04-06 00:00:00  FR              Apartment        Entire home/apt  Real Bed               

Кодируем признаки

In [75]:
cat_col = ["City", "Zipcode", "Last scraped", "Country code", "Property type", "Room type", "Bed type"]

In [76]:
for cc in cat_col:
    df[cc] = df[cc].ascharacter().asfactor()

In [78]:
fcolumns = ["City", "Zipcode", "Last scraped", "Country code", "Property type", "Room type", "Bed type", "Accommodates", "Bathrooms", "Bedrooms"]
target = "Price"

In [79]:
train, valid= df.split_frame(ratios = [.7], seed = 7665)

In [80]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator

gbm = H2OGradientBoostingEstimator(nbins_cats=2048, nbins=64, ntrees=13, learn_rate=0.09, seed=7896)

In [81]:
gbm.train(x = fcolumns, y = target, training_frame = train, validation_frame = valid)

gbm Model Build progress: [###############################################] 100%

In [82]:
gbm.r2(valid=True)

0.6234217428516223

Обучение прошло очень быстро, а полученное качество весьма неплохо для данной задачи!

В конце попробуем поиграться с еще одной реализацией распределенного градиентного бустинга. В этот раз от компании Microsoft. В наболе MMLSpark, с которым мы с вами уже работали, если инструмент LightGBM.

In [1]:
%%configure -f
{
    "name": "mmlspark",
    "conf": {
        "spark.jars.packages": "com.microsoft.ml.spark:mmlspark_2.11:1.0.0-rc3",
        "spark.jars.repositories": "https://mmlspark.azureedge.net/maven",
        "spark.jars.excludes": "org.scala-lang:scala-reflect,org.apache.spark:spark-tags_2.11,org.scalactic:scalactic_2.11,org.scalatest:scalatest_2.11"
    }
}

In [2]:
data = spark.read.option("delimiter", "\t").csv('/airbnb/data.tsv', header=True, inferSchema=True)
train, test = (
    data
    .na.drop('any')
    .withColumn('PriceN', data.Price.cast('integer'))
    .randomSplit([0.9, 0.1], 422)
)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7,application_1614009348742_0011,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.
/usr/bin/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')

In [3]:
train.show()

+-----+---------------+-------+------------+------------+-------------+---------------+--------+------------+---------+--------+------+
|Price|           City|Zipcode|Last scraped|Country code|Property type|      Room type|Bed type|Accommodates|Bathrooms|Bedrooms|PriceN|
+-----+---------------+-------+------------+------------+-------------+---------------+--------+------------+---------+--------+------+
|  0.0|         Austin|  78741|  2017-03-07|          US|    Apartment|   Private room|Real Bed|         2.0|      1.0|     1.0|     0|
|  0.0|         Berlin|  13353|  2017-05-08|          DE|    Apartment|   Private room|Real Bed|         2.0|      1.0|     1.0|     0|
|  0.0|          Bronx|  10460|  2017-10-02|          US|    Apartment|   Private room|Real Bed|         2.0|      1.5|     1.0|     0|
|  0.0|          Bronx|  10461|  2017-10-03|          US|        House|Entire home/apt|Real Bed|         4.0|      1.0|     0.0|     0|
|  0.0|       Brooklyn|  11203|  2017-10-02|    

Для простоты, закодируем признаки через Hashing Trick, используя VWFeaturizer

In [4]:
from mmlspark.vw import VowpalWabbitFeaturizer
fcolumns = ["City", "Zipcode", "Last scraped", "Country code", "Property type", "Room type", "Bed type", "Accommodates", "Bathrooms", "Bedrooms"]
vw_featurizer = VowpalWabbitFeaturizer(
    inputCols=fcolumns, 
    outputCol="features",
    numBits=20
)

In [5]:
train = vw_featurizer.transform(train)
test = vw_featurizer.transform(test)

In [6]:
train.rdd.first()

Row(Price=u'0.0', City=u'Austin', Zipcode=u'78741', Last scraped=u'2017-03-07', Country code=u'US', Property type=u'Apartment', Room type=u'Private room', Bed type=u'Real Bed', Accommodates=2.0, Bathrooms=1.0, Bedrooms=1.0, PriceN=0, features=SparseVector(1048576, {137222: 1.0, 161166: 1.0, 186238: 1.0, 268028: 1.0, 462415: 1.0, 584524: 1.0, 638668: 1.0, 645771: 1.0, 649339: 2.0, 960043: 1.0}))

Запускаем обучаться LightGBM распределенно на кластере. Здесь также можно подкрутить количество корзинок, которое будет использоваться при обучении.

In [7]:
from mmlspark.lightgbm import LightGBMRegressor
model = LightGBMRegressor(
    objective='quantile',
    maxBin=255,
    labelCol='PriceN',
    alpha=0.2,
    learningRate=0.3
)

In [8]:
model = model.fit(train)

In [9]:
dir(model)

['__abstractmethods__', '__class__', '__del__', '__delattr__', '__dict__', '__doc__', '__format__', '__getattribute__', '__hash__', '__init__', '__metaclass__', '__module__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_cache', '_abc_negative_cache', '_abc_negative_cache_version', '_abc_registry', '_call_java', '_clear', '_copyValues', '_copy_params', '_create_from_java_class', '_create_params_from_java', '_defaultParamMap', '_dummy', '_empty_java_param_map', '_from_java', '_java_obj', '_make_java_param_pair', '_new_java_array', '_new_java_obj', '_paramMap', '_params', '_randomUID', '_resetUid', '_resolveParam', '_set', '_setDefault', '_shouldOwn', '_to_java', '_transfer_param_map_from_java', '_transfer_param_map_to_java', '_transfer_params_from_java', '_transfer_params_to_java', '_transform', 'copy', 'explainParam', 'explainParams', 'extractParamMap', 'getFeatureImportances', 'getFeatureShaps', '

In [10]:
from mmlspark.train import ComputeModelStatistics

prediction = model.transform(test)

metrics = ComputeModelStatistics(
    evaluationMetric='regression',
    labelCol='PriceN',
    scoresCol='prediction'
).transform(prediction)

DataFrame[mean_squared_error: double, root_mean_squared_error: double, R^2: double, mean_absolute_error: double]

In [11]:
metrics.show()

+------------------+-----------------------+------------------+-------------------+
|mean_squared_error|root_mean_squared_error|               R^2|mean_absolute_error|
+------------------+-----------------------+------------------+-------------------+
| 9158.410046390361|      95.69958226863042|0.5477662708112963|  50.27381583977375|
+------------------+-----------------------+------------------+-------------------+